# 🎉 Out-of-Distribution (OOD) with PCA using Deep Features from the Latent Space

The goal of this notebook is to understand the depths of using Principal Component Analysis in order to perform OOD tasks using deep features from the latent space

## 📝 Plan of action

### ♻️ Preprocessing phase

In order to achieve our goal, we need to understand how the dataset is structured.

For this notebook, we are going to use the CBIR 15 dataset, that contains images of different places, such as an office, a bedroom, a mountain, etc. Note that there are some places that are similar one to another, i.e. a bedroom and a living room.

Thus, in order to extract the features of the images we have to preprocess those images:

- Get the images that are located in data/CBIR_15-scene and fit them to a dataframe using Pandas
  - Locate the "Labels.txt" file: it shows where the indexes of the images from each category starts
- Create the dataset with this information with two columns: the path to the image and its category
- Transform all of the images in the same size (in this case, we are going with 256x256)
  
Now, in order to extract the features, it's necessary to divide the reshaped images into patches of 32x32 pixels. This is good to perform processing tasks to avoid waiting long periods of time.

After all the preprocess, we should separate the images into two different foldes: one contains the patches of the training images that is going to give us their principal components and dimensions, and the other is the patches of the test images, that is going to be tested to fit into those dimensions and we'll get an OOD score afterwards.

### 🏋🏽‍♂️ Training phase

With the images that are stored inside the "patches_train" folder, the first thing we are going to do is _normalize_ all of the images to find the correct maximum covariance and transforming all the variables into the same scale.

Next, we should then apply the PCA with all the components. As we have patches of 32x32, we'll be having 1024 features, hence components. Then we plot a graph to see how many components truly contributes for the most variance of the data - and give us more information about it. We're going to take the threshold of 95% of variance in this notebook.

After getting the PCA with components that describe 95% of the variance, it's time to test our images and see how far of the residual space their data can be found.

### ⚗️ Test phase and results

In this phase, we take the test images and normalize then with the same scale of each PCA. This is important to maintain consistency throughout the final results and measure the norms in the new dimension properly.

After that, we calculate the norm of the projection of the given data into the orthogonal space of the principal component and divide it by the norm of the data in relation to the origin. This is the OOD score.

We calculate the mean of the score for each category and get the minimal one. The current environment is the smallest.


--------------------------

First of all, we need to understand which libraries we are going to use:

- os: Deals with the operation system interface such as finding the relative and absolute path of files inside a project and reading/writing files for example.
- sys: This module provides access to some variables used or maintained by the interpreter and to functions that interact strongly with the interpreter.
- numpy: NumPy is the fundamental package for scientific computing in Python. It is a Python library that provides a multidimensional array object, various derived objects (such as masked arrays and matrices), and an assortment of routines for fast operations on arrays, including mathematical, logical, shape manipulation, sorting, selecting, I/O, discrete Fourier transforms, basic linear algebra, basic statistical operations, random simulation and much more.
- pandas: Pandas is an open source, BSD-licensed library providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language.
- matplotlib: Deals with plotting graphs to visualize data in a graphical way.
- sklearn: Scikit-learn provides dozens of built-in machine learning algorithms and models, called estimators.

In [17]:
import os
import sys
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


I'd suggest to use a conda virtual environment in order to avoid messing up your base kernel environment and causing dependency errors in the future.

After you successfully installed all the modules, it's time to import our custom modules that are going to deal with:

- Creation of our dataframe using pandas
- Separation of our dataset into patches of 32x32 in folders of training and test

In [18]:

sys.path.append(os.path.abspath('..'))

from dataframe_generator import *
from images_standardizing import *

In [19]:
import tarfile

def extract_tgz(tgz_path, extract_to):
    if not os.path.exists(extract_to):
        os.makedirs(extract_to)
    
    with tarfile.open(tgz_path, 'r:gz') as tar:
        tar.extractall(path=extract_to)
        print(f"Arquivos extraídos para {extract_to}")

tgz_path = '../CBIR_15-Scene.tgz'
extract_to = '../data/'

extract_tgz(tgz_path, extract_to)

Arquivos extraídos para ../data/


In [20]:
df = create_dataframe()
df

                             image_path category
0        ../data/CBIR_15-Scene/00/1.jpg  Bedroom
1        ../data/CBIR_15-Scene/00/2.jpg  Bedroom
2        ../data/CBIR_15-Scene/00/3.jpg  Bedroom
3        ../data/CBIR_15-Scene/00/4.jpg  Bedroom
4        ../data/CBIR_15-Scene/00/5.jpg  Bedroom
...                                 ...      ...
4480  ../data/CBIR_15-Scene/14/4481.jpg    Store
4481  ../data/CBIR_15-Scene/14/4482.jpg    Store
4482  ../data/CBIR_15-Scene/14/4483.jpg    Store
4483  ../data/CBIR_15-Scene/14/4484.jpg    Store
4484  ../data/CBIR_15-Scene/14/4485.jpg    Store

[4485 rows x 2 columns]


,image_path,category
0,../data/CBIR_15-Scene/00/1.jpg,Bedroom
1,../data/CBIR_15-Scene/00/2.jpg,Bedroom
2,../data/CBIR_15-Scene/00/3.jpg,Bedroom
3,../data/CBIR_15-Scene/00/4.jpg,Bedroom
4,../data/CBIR_15-Scene/00/5.jpg,Bedroom
...,...,...
4480,../data/CBIR_15-Scene/14/4481.jpg,Store
4481,../data/CBIR_15-Scene/14/4482.jpg,Store
4482,../data/CBIR_15-Scene/14/4483.jpg,Store
4483,../data/CBIR_15-Scene/14/4484.jpg,Store


## ☝️ Part I: Comparing two different environments

### ♻️ Preprocessing phase

Now we start our experiments to understand if our idea work, however this time we are going to understand what happens with our approach using two different environments.

In our case, I'm going to take the **Coast** and **Office** environments arbitrarily.


In [21]:
train_categories = ['Coast', 'Office']

df_different = df[df['category'].isin(train_categories)]
df_different

,image_path,category
1267,../data/CBIR_15-Scene/05/1268.jpg,Coast
1268,../data/CBIR_15-Scene/05/1269.jpg,Coast
1269,../data/CBIR_15-Scene/05/1270.jpg,Coast
1270,../data/CBIR_15-Scene/05/1271.jpg,Coast
1271,../data/CBIR_15-Scene/05/1272.jpg,Coast
...,...,...
4165,../data/CBIR_15-Scene/13/4166.jpg,Office
4166,../data/CBIR_15-Scene/13/4167.jpg,Office
4167,../data/CBIR_15-Scene/13/4168.jpg,Office
4168,../data/CBIR_15-Scene/13/4169.jpg,Office


It's time to separate our dataset into train and test. We should use the built-in function of sklearn to do this:

In [26]:
X = df_different['image_path'].tolist()
y = df_different['category'].tolist()
unique_categories = list(df_different['category'].unique())
print(f"Unique categories: {unique_categories}")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

standard_size = (224, 224)

Unique categories: ['Coast', 'Office']


Making sure that everything went well, we plot the grid of all the patches from the first image of our training set

This is exactly what the module that's inside our "image_patching.py" do. So we now, need to save everything into the subfolders by calling that function:

In [27]:
create_images_set(X_train, X_test, y_train, y_test, output_dir_train='images_train', output_dir_test='images_test', standard_size=standard_size)

Now, we should load our patches for training:

In [28]:
#training_images_by_category = load_images_by_category('images_train', y, image_size=(224, 224))
training_images_by_category = load_images_by_category('images_train', unique_categories, image_size=(224, 224))
print(training_images_by_category)

Loading images from images_train/Coast
Loaded 288 images for category Coast
Loading images from images_train/Office
Loaded 172 images for category Office
{'Coast': array([[[[105, 105, 105],
         [105, 105, 105],
         [110, 110, 110],
         ...,
         [ 69,  69,  69],
         [ 70,  70,  70],
         [ 70,  70,  70]],

        [[ 95,  95,  95],
         [ 98,  98,  98],
         [105, 105, 105],
         ...,
         [ 67,  67,  67],
         [ 68,  68,  68],
         [ 67,  67,  67]],

        [[ 96,  96,  96],
         [ 96,  96,  96],
         [ 98,  98,  98],
         ...,
         [ 67,  67,  67],
         [ 67,  67,  67],
         [ 66,  66,  66]],

        ...,

        [[ 17,  17,  17],
         [ 16,  16,  16],
         [ 18,  18,  18],
         ...,
         [ 48,  48,  48],
         [ 46,  46,  46],
         [ 44,  44,  44]],

        [[ 40,  40,  40],
         [ 33,  33,  33],
         [ 25,  25,  25],
         ...,
         [ 51,  51,  51],
         [ 50,  

In [29]:
from sklearn.preprocessing import StandardScaler

def normalize_and_center_images(images, scaler=None):
    # Flatten the images array for scaling
    num_images, height, width, channels = images.shape
    flattened_images = images.reshape((num_images, -1))
    
    if scaler is None:
        scaler = StandardScaler()
        standardized_flattened_images = scaler.fit_transform(flattened_images)
    else:
        standardized_flattened_images = scaler.transform(flattened_images)
    
    # Reshape back to the original shape
    standardized_images = standardized_flattened_images.reshape((num_images, height, width, channels))
    return standardized_images, scaler

standardized_images_by_category = {}
scalers_by_category = {}
for category, images in training_images_by_category.items():
    print(images.shape)
    standardized_images, scaler = normalize_and_center_images(images)
    standardized_images_by_category[category] = standardized_images
    scalers_by_category[category] = scaler
    print(f"Category {category}, images shape: {standardized_images.shape}")


(288, 224, 224, 3)
Category Coast, images shape: (288, 224, 224, 3)
(172, 224, 224, 3)
Category Office, images shape: (172, 224, 224, 3)


### 🏋🏽‍♂️ Training phase

Now that the have our training patches stored in that variable above, we should start our analysis with PCA.

First of all, we **need to normalize and center** the data. It's so importantt that I had to emphasize it. Plus, since we are dealing with different categories, each one of them should be normalized with a different scaler (and we're going to save it for later).

In [30]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model

In [31]:
preprocessed_images_by_category = {}
input_size = (224, 224)
for category, images in training_images_by_category.items():
    resized_images = np.array([resize_image(patch, input_size) for patch in images])
    preprocessed_images = preprocess_input(resized_images)
    preprocessed_images_by_category[category] = preprocessed_images

for category, images in preprocessed_images_by_category.items():
    print(f"Category {category}, images shape: {images.shape}")

Category Coast, images shape: (288, 224, 224, 3)
Category Office, images shape: (172, 224, 224, 3)


In [32]:
preprocessed_images_by_category = standardized_images_by_category 

base_model = VGG16(weights='imagenet', include_top=True)
base_model.summary()


Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [33]:
# Getting the before last layer (Fully connected)
model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc2').output)

In [34]:
features_by_category = {}
for category, images in preprocessed_images_by_category.items():
    features = model.predict(images)
    features_by_category[category] = features

for category, features in features_by_category.items():
    print(f"Category {category}, features shape: {features.shape}")

6/6 [==============================] - 15s 2s/step
Category Coast, features shape: (288, 4096)
Category Office, features shape: (172, 4096)


In [35]:
from sklearn.decomposition import PCA

pca_by_category = {}
explained_variance_by_category = {}

for category, features in features_by_category.items():
    pca = PCA(n_components=0.95)  
    principal_components = pca.fit_transform(features)
    pca_by_category[category] = pca
    explained_variance_by_category[category] = pca.explained_variance_ratio_
    
    print(f"Category {category}, principal components: {principal_components.shape[1]}")

for category, pca in pca_by_category.items():
    print(f"Category {category}, principal components shape: {pca.components_.shape}")
    print(f"Category {category}, explained variance: {np.sum(explained_variance_by_category[category]) * 100:.2f}%")


Category Coast, principal components: 63
Category Office, principal components: 68
Category Coast, principal components shape: (63, 4096)
Category Coast, explained variance: 95.10%
Category Office, principal components shape: (68, 4096)
Category Office, explained variance: 95.11%


### Testing phase


In [38]:
def load_and_preprocess_test_images(test_dir, categories, image_size, input_size):
    test_images_by_category = load_images_by_category(test_dir, categories, image_size)
    test_standardized_images_by_category = {}
    test_scalers_by_category = {}

    for category, images in test_images_by_category.items():
        test_standardized_images, test_scaler = normalize_and_center_images(images)
        test_standardized_images_by_category[category] = test_standardized_images
        test_scalers_by_category[category] = test_scaler

    # test_preprocessed_images_by_category = {}
    # for category, images in test_standardized_images_by_category.items():
    #     test_resized_images = np.array([resize_image(image, input_size) for image in images])
    #     test_preprocessed_images = preprocess_input(test_resized_images)
    #     test_preprocessed_images_by_category[category] = test_preprocessed_images

    return test_standardized_images_by_category

image_size = (224, 224)

test_preprocessed_images_by_category = load_and_preprocess_test_images('images_test', y, image_size, input_size)


Loading images from images_test/Coast
Loaded 72 images for category Coast
Loading images from images_test/Coast
Loaded 72 images for category Coast
Loading images from images_test/Coast
Loaded 72 images for category Coast
Loading images from images_test/Coast
Loaded 72 images for category Coast
Loading images from images_test/Coast
Loaded 72 images for category Coast
Loading images from images_test/Coast
Loaded 72 images for category Coast
Loading images from images_test/Coast
Loaded 72 images for category Coast
Loading images from images_test/Coast
Loaded 72 images for category Coast
Loading images from images_test/Coast
Loaded 72 images for category Coast
Loading images from images_test/Coast
Loaded 72 images for category Coast
Loading images from images_test/Coast
Loaded 72 images for category Coast
Loading images from images_test/Coast
Loaded 72 images for category Coast
Loading images from images_test/Coast
Loaded 72 images for category Coast
Loading images from images_test/Coast


In [39]:
def extract_features_with_vgg16(model, preprocessed_images_by_category):
    features_by_category = {}
    for category, images in preprocessed_images_by_category.items():
        features = model.predict(images)
        features_by_category[category] = features
    return features_by_category

test_features_by_category = extract_features_with_vgg16(model, test_preprocessed_images_by_category)


2/2 [==============================] - 3s 853ms/step


In [41]:
def calculate_reconstruction_error(test_features, pca_by_category):
    reconstruction_errors_by_category = {}
    mean_reconstruction_errors_by_category = {}
    
    for category, pca in pca_by_category.items():
        principal_components = pca.transform(test_features)
        
        reconstructed_features = pca.inverse_transform(principal_components)
        reconstruction_error= np.linalg.norm(test_features - reconstructed_features)
        reconstruction_errors_by_category[category] = reconstruction_error

    for category in reconstruction_errors_by_category:
        mean_reconstruction_errors_by_category[category] = np.mean(reconstruction_errors_by_category[category])
    
    best_category = min(mean_reconstruction_errors_by_category, key=mean_reconstruction_errors_by_category.get)

    for category in mean_reconstruction_errors_by_category:
        print(f"Category {category}, mean reconstruction error: {mean_reconstruction_errors_by_category[category]}")
    
    print(f"Best category: {best_category}")

    return mean_reconstruction_errors_by_category, best_category

for category, test_features in test_features_by_category.items():
    print(f"Test category: {category}")
    mean_reconstruction_errors, best_category = calculate_reconstruction_error(test_features, pca_by_category)


Test category: Coast
Category Coast, mean reconstruction error: 17.04598617553711
Category Office, mean reconstruction error: 71.71232604980469
Best category: Coast
Test category: Office
Category Coast, mean reconstruction error: 52.81996536254883
Category Office, mean reconstruction error: 22.01555061340332
Best category: Office


## ✌️ Part II: Comparing two similar environments

In [42]:
train_categories = ['Bedroom', 'LivingRoom']

df_different = df[df['category'].isin(train_categories)]
df_different

,image_path,category
0,../data/CBIR_15-Scene/00/1.jpg,Bedroom
1,../data/CBIR_15-Scene/00/2.jpg,Bedroom
2,../data/CBIR_15-Scene/00/3.jpg,Bedroom
3,../data/CBIR_15-Scene/00/4.jpg,Bedroom
4,../data/CBIR_15-Scene/00/5.jpg,Bedroom
...,...,...
1262,../data/CBIR_15-Scene/04/1263.jpg,LivingRoom
1263,../data/CBIR_15-Scene/04/1264.jpg,LivingRoom
1264,../data/CBIR_15-Scene/04/1265.jpg,LivingRoom
1265,../data/CBIR_15-Scene/04/1266.jpg,LivingRoom


In [46]:
X = df_different['image_path']
y = df_different['category']
(X_train, X_test, y_train, y_test) = train_test_split(X, y, test_size=0.2, random_state=10)

image_size = (224, 224)
unique_categories = list(df_different['category'].unique())
print(f"Unique categories: {unique_categories}")


Unique categories: ['Bedroom', 'LivingRoom']


In [48]:
create_images_set(X_train, X_test, y_train, y_test, output_dir_train='images_train', output_dir_test='images_test', standard_size=standard_size)

In [49]:
training_images_by_category = load_images_by_category('images_train', unique_categories, image_size=(224, 224))
print(training_images_by_category)

Loading images from images_train/Bedroom
Loaded 173 images for category Bedroom
Loading images from images_train/LivingRoom
Loaded 231 images for category LivingRoom
{'Bedroom': array([[[[218, 218, 218],
         [222, 222, 222],
         [224, 224, 224],
         ...,
         [210, 210, 210],
         [225, 225, 225],
         [225, 225, 225]],

        [[195, 195, 195],
         [198, 198, 198],
         [197, 197, 197],
         ...,
         [235, 235, 235],
         [255, 255, 255],
         [255, 255, 255]],

        [[168, 168, 168],
         [162, 162, 162],
         [168, 168, 168],
         ...,
         [238, 238, 238],
         [255, 255, 255],
         [254, 254, 254]],

        ...,

        [[174, 174, 174],
         [182, 182, 182],
         [183, 183, 183],
         ...,
         [183, 183, 183],
         [185, 185, 185],
         [204, 204, 204]],

        [[192, 192, 192],
         [196, 196, 196],
         [195, 195, 195],
         ...,
         [249, 249, 249],
  

In [51]:
standardized_images_by_category = {}
scalers_by_category = {}
for category, images in training_images_by_category.items():
    standardized_images, scaler = normalize_and_center_images(images)
    standardized_images_by_category[category] = standardized_images
    scalers_by_category[category] = scaler


In [52]:
preprocessed_images_by_category = {}
for category, images in standardized_images_by_category.items():
    resized_images = np.array([resize_image(image, input_size) for image in images])
    preprocessed_images = preprocess_input(resized_images)
    preprocessed_images_by_category[category] = preprocessed_images

preprocessed_images = standardized_images_by_category

for category, images in preprocessed_images_by_category.items():
    print(f"Categoria {category}, images shape: {images.shape}")

Categoria Bedroom, images shape: (173, 224, 224, 3)
Categoria LivingRoom, images shape: (231, 224, 224, 3)


In [53]:
base_model = VGG16(weights='imagenet', include_top=True)

In [54]:
# Getting the before last layer (Fully connected)
model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc2').output)

In [55]:
features_by_category = {}
for category, images in preprocessed_images_by_category.items():
    features = model.predict(images)
    features_by_category[category] = features

for category, features in features_by_category.items():
    print(f"Category {category}, features shape: {features.shape}")

8/8 [==============================] - 20s 2s/step
Category Bedroom, features shape: (173, 4096)
Category LivingRoom, features shape: (231, 4096)


In [56]:
from sklearn.decomposition import PCA

pca_by_category = {}
explained_variance_by_category = {}

for category, features in features_by_category.items():
    pca = PCA(n_components=0.95)  
    principal_components = pca.fit_transform(features)
    pca_by_category[category] = pca
    explained_variance_by_category[category] = pca.explained_variance_ratio_
    
    print(f"Category {category}, principal components: {principal_components.shape[1]}")

for category, pca in pca_by_category.items():
    print(f"Category {category}, principal components shape: {pca.components_.shape}")
    print(f"Category {category}, explained variance: {np.sum(explained_variance_by_category[category]) * 100:.2f}%")


Category Bedroom, principal components: 41
Category LivingRoom, principal components: 41
Category Bedroom, principal components shape: (41, 4096)
Category Bedroom, explained variance: 95.02%
Category LivingRoom, principal components shape: (41, 4096)
Category LivingRoom, explained variance: 95.02%


In [57]:
def load_and_preprocess_test_images(test_dir, categories, image_size, input_size):
    test_images_by_category = load_images_by_category(test_dir, categories, image_size)
    test_standardized_images_by_category = {}
    test_scalers_by_category = {}

    for category, images in test_images_by_category.items():
        test_standardized_images, test_scaler = normalize_and_center_images(images)
        test_standardized_images_by_category[category] = test_standardized_images
        test_scalers_by_category[category] = test_scaler

    return test_standardized_images_by_category

test_preprocessed_images_by_category = load_and_preprocess_test_images('images_test', y, image_size, input_size)


Loading images from images_test/Bedroom
Loaded 43 images for category Bedroom
Loading images from images_test/Bedroom
Loaded 43 images for category Bedroom
Loading images from images_test/Bedroom
Loaded 43 images for category Bedroom
Loading images from images_test/Bedroom
Loaded 43 images for category Bedroom
Loading images from images_test/Bedroom
Loaded 43 images for category Bedroom
Loading images from images_test/Bedroom
Loaded 43 images for category Bedroom
Loading images from images_test/Bedroom
Loaded 43 images for category Bedroom
Loading images from images_test/Bedroom
Loaded 43 images for category Bedroom
Loading images from images_test/Bedroom
Loaded 43 images for category Bedroom
Loading images from images_test/Bedroom
Loaded 43 images for category Bedroom
Loading images from images_test/Bedroom
Loaded 43 images for category Bedroom
Loading images from images_test/Bedroom
Loaded 43 images for category Bedroom
Loading images from images_test/Bedroom
Loaded 43 images for cat

In [58]:
def extract_features_with_vgg16(model, preprocessed_images_by_category):
    features_by_category = {}
    for category, images in preprocessed_images_by_category.items():
        features = model.predict(images)
        features_by_category[category] = features
    return features_by_category

test_features_by_category = extract_features_with_vgg16(model, test_preprocessed_images_by_category)


2/2 [==============================] - 5s 2s/step


In [59]:
def calculate_reconstruction_error(test_features, pca_by_category):
    reconstruction_errors_by_category = {}
    mean_reconstruction_errors_by_category = {}
    
    for category, pca in pca_by_category.items():
        principal_components = pca.transform(test_features)
        
        reconstructed_features = pca.inverse_transform(principal_components)
        reconstruction_error= np.linalg.norm(test_features - reconstructed_features)
        reconstruction_errors_by_category[category] = reconstruction_error

    for category in reconstruction_errors_by_category:
        mean_reconstruction_errors_by_category[category] = np.mean(reconstruction_errors_by_category[category])
    
    best_category = min(mean_reconstruction_errors_by_category, key=mean_reconstruction_errors_by_category.get)

    for category in mean_reconstruction_errors_by_category:
        print(f"Category {category}, mean reconstruction error: {mean_reconstruction_errors_by_category[category]}")
    
    print(f"Best category: {best_category}")

    return mean_reconstruction_errors_by_category, best_category

for category, test_features in test_features_by_category.items():
    print(f"Test category: {category}")
    mean_reconstruction_errors, best_category = calculate_reconstruction_error(test_features, pca_by_category)


Test category: Bedroom
Category Bedroom, mean reconstruction error: 250.82421875
Category LivingRoom, mean reconstruction error: 243.07977294921875
Best category: LivingRoom
Test category: LivingRoom
Category Bedroom, mean reconstruction error: 286.8707275390625
Category LivingRoom, mean reconstruction error: 278.4144287109375
Best category: LivingRoom
